# INITIALISING DB

In [ ]:
def init_connection():
    """Create a database connection that persists across Streamlit reruns"""
    try:
        # Runs locally
        engine = create_engine('sqlite:///database.db') 
        # Test the connection
        with engine.connect() as conn:
            conn.execute(text("SELECT 1"))
        logger.info("Database connection successful")
        return engine
    except Exception as e:
        logger.error(f"Error connecting to database: {e}")
        st.error("Failed to connect to database")
        return None